# DuckDB on Python
- Let's try question 2 using python & duckdb
- duckdb works with other languages like R, Java and many more

In [26]:
pip install duckdb


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import duckdb
import pandas as pd

In [28]:
df = pd.read_csv('/workspaces/PracticeSQL/datasets/partytime.csv')
# duckdb has also has a read_csv function
# but we want to show how to use it with pandas dataframes

In [29]:
help(duckdb.connect)
# Connect to DuckDB
con = duckdb.connect(database=':memory:')

Help on built-in function connect in module duckdb.duckdb:

connect(...) method of builtins.PyCapsule instance
    connect(database: object = ':memory:', read_only: bool = False, config: dict = None) -> duckdb.DuckDBPyConnection

    Create a DuckDB database instance. Can take a database file name to read/write persistent data and a read_only flag if no changes are desired



In [30]:
query = """
SELECT *, DAYOFWEEK(Date::DATE) as DOW,
    CASE WHEN DOW IN (0, 6) THEN TRUE ELSE FALSE END as IsWeekend,
    CASE WHEN ISWeekend AND MOOD >= 4 THEN TRUE ELSE FALSE END as PartyTime
    FROM df;
"""
con.execute(query).df()  # Execute the query and return the result as a DataFrame

,Date,Mood,DOW,IsWeekend,PartyTime
0,23-03-01,4,3,False,False
1,23-03-02,3,4,False,False
2,23-03-03,5,5,False,False
3,23-03-04,2,6,True,False
4,23-03-05,4,0,True,True
5,23-03-06,5,1,False,False
6,23-03-07,3,2,False,False
7,23-03-08,4,3,False,False
8,23-03-09,4,4,False,False
9,23-03-10,5,5,False,False


In [31]:
duckdb.write_csv(con.execute(query).df(), '/workspaces/PracticeSQL/datasets/partytime_o.csv')

### Let's try using duckdb.read_csv now

In [32]:
duckdb.read_csv('/workspaces/PracticeSQL/datasets/partytime.csv')  # Read the output CSV to verify
duckdb.sql("select * from '/workspaces/PracticeSQL/datasets/partytime.csv'").df()

,Date,Mood
0,2023-03-01,4
1,2023-03-02,3
2,2023-03-03,5
3,2023-03-04,2
4,2023-03-05,4
5,2023-03-06,5
6,2023-03-07,3
7,2023-03-08,4
8,2023-03-09,4
9,2023-03-10,5


# using SQL directly

In [33]:
duckdb.sql("select * from read_csv_auto('/workspaces/PracticeSQL/datasets/partytime.csv')").df()

,Date,Mood
0,2023-03-01,4
1,2023-03-02,3
2,2023-03-03,5
3,2023-03-04,2
4,2023-03-05,4
5,2023-03-06,5
6,2023-03-07,3
7,2023-03-08,4
8,2023-03-09,4
9,2023-03-10,5


In [34]:
# close the connection as best practice
con.close()

In [35]:
# create a connection to a file called 'file.db'
new_con = duckdb.connect("file.db")

In [36]:
duckdb.sql("""
CREATE OR REPLACE MACRO IS_WEEKEND(DATE) AS
  EXTRACT(DAYOFWEEK FROM DATE::DATE) IN (0,6);
""", connection=new_con)


In [37]:
df = pd.read_csv('/workspaces/PracticeSQL/datasets/partytime.csv')
query = """
SELECT IS_WEEKEND(Date) AS IsWeekend,
MOOD,
CASE WHEN IS_WEEKEND(Date) AND MOOD >= 4 THEN TRUE ELSE FALSE END AS PartyTime
FROM df;
"""
duckdb.sql(query, connection=new_con).df()

,IsWeekend,Mood,PartyTime
0,False,4,False
1,False,3,False
2,False,5,False
3,True,2,False
4,True,4,True
5,False,5,False
6,False,3,False
7,False,4,False
8,False,4,False
9,False,5,False


In [38]:
new_con.close()